In [2]:
import duckdb
import pandas as pd

team_defense_snaps = pd.read_csv('intermediate_content/team_defense_snaps.csv')  
team_offense_snaps = pd.read_csv('intermediate_content/team_offense_snaps.csv')
transfer_player_career_wicks = pd.read_csv('intermediate_content/transfer_player_career_wicks.csv')
transfer_player_snap_counts = pd.read_csv('intermediate_content/transfer_player_snap_counts.csv')
transfer_player_career_wicks

,season,position,pff_player_id,wick_score,capped_wick_score
0,2024,QB,149182,52.7,44.4
1,2024,QB,105568,65.0,60.5
2,2024,QB,158683,45.7,35.3
3,2024,QB,122162,50.3,41.3
4,2024,QB,129906,53.6,45.6
...,...,...,...,...,...
20329,2025,S,157707,54.2,54.2
20330,2025,S,145579,46.9,46.9
20331,2025,S,123103,50.7,50.7
20332,2025,S,170434,39.5,39.5


In [3]:
# find players who go up in some way (fcs to group of 6, group of 6 to power 4)

moved_up_df = transfer_player_snap_counts[
    (
        (transfer_player_snap_counts["prev_level"] == "FCS") &
        (transfer_player_snap_counts["current_level"].isin(["Power Four", "Group of Six"]))
    )
    |
    (
        (transfer_player_snap_counts["prev_level"] == "Group of Six") &
        (transfer_player_snap_counts["current_level"] == "Power Four")
    )
]
# moved_up_df

In [37]:
# find players who succeeded in their moved up schools

succeeded_df = moved_up_df[
    (moved_up_df["current_def_snaps"] > 100) |
    (moved_up_df["current_off_snaps"] > 100)
]

succeeded_df.to_csv("succeeded.csv", index=False)

succeeded_df

,pff_player_id,player_name,prev_season,current_season,prev_position,current_pos,prev_gsis,prev_school,current_gsis,current_school,prev_level,current_level,prev_def_snaps,current_def_snaps,prev_off_snaps,current_off_snaps
0,21922,Mark Williams,2020,2021,CB,CB,ILEA,Eastern Illinois,LANE,Louisiana-Monroe,FCS,Group of Six,348.0,303.0,NaN,NaN
7,23441,Reggie Stubblefield,2020,2021,CB,S,TXPV,Prairie View A&M,KSST,Kansas State,FCS,Power Four,185.0,501.0,NaN,NaN
13,28919,Jawon Hamilton,2020,2021,HB,HB,VAJM,James Madison,MIEA,Eastern Michigan,FCS,Group of Six,NaN,NaN,170.0,227.0
16,30285,Colby Reeder,2021,2022,LB,LB,DEUN,Delaware,IAST,Iowa State,FCS,Power Four,441.0,361.0,NaN,NaN
17,30557,Adam Cofield,2020,2021,HB,HB,NDST,North Dakota State,KYWE,Western Kentucky,FCS,Group of Six,NaN,NaN,41.0,390.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15130,189285,Ayden Jones,2024,2025,LB,LB,TXPV,Prairie View A&M,TXSW,Texas State,FCS,Group of Six,370.0,113.0,NaN,NaN
15220,190753,Joseph Williams,2024,2025,WR,WR,OKTU,Tulsa,COUN,Colorado,Group of Six,Power Four,NaN,NaN,490.0,128.0
15271,192037,Anthony Palano,2024,2025,LB,LB,SDST,South Dakota State,WAST,Washington State,FCS,Group of Six,NaN,103.0,NaN,NaN
15276,192135,Baylor Hayes,2024,2025,QB,QB,TNEA,East Tennessee State,OKTU,Tulsa,FCS,Group of Six,NaN,NaN,80.0,110.0


In [5]:
query_1 = duckdb.query("""

SELECT prev_position, count(*) count
FROM succeeded_df
GROUP BY prev_position
ORDER BY count
""").to_df()
query_1

,prev_position,count
0,FB,2
1,C,45
2,QB,59
3,HB,64
4,TE,75
5,G,82
6,DI,111
7,LB,122
8,S,128
9,T,137


In [6]:
succeeded_expanded_df_raw = duckdb.query("""

SELECT transfer_player_career_wicks.pff_player_id, player_name, prev_season, current_season, wick_score, capped_wick_score, prev_position, current_pos, prev_school, current_school, prev_def_snaps, current_def_snaps, prev_off_snaps, current_off_snaps
FROM transfer_player_career_wicks JOIN succeeded.csv succeeded ON transfer_player_career_wicks.pff_player_id = succeeded.pff_player_id
WHERE current_pos = prev_position
""").to_df()

# succeeded_expanded_df_raw

In [7]:
succeeded_expanded_df = duckdb.query("""

SELECT pff_player_id, player_name, prev_season, current_season, prev_position, current_pos, prev_school, current_school, 
        round(avg(wick_score), 1) wick_score, round(avg(capped_wick_score), 1) capped_wick_score, avg(prev_def_snaps) prev_def_snaps, 
        avg(current_def_snaps) current_def_snaps, avg(prev_off_snaps) prev_off_snaps, avg(current_off_snaps) current_off_snaps
FROM succeeded_expanded_df_raw
GROUP BY pff_player_id, player_name, prev_season, current_season, prev_position, current_pos, prev_school, current_school
""").to_df()

# succeeded_expanded_df

succeeded_def_df = succeeded_expanded_df[(succeeded_expanded_df["current_def_snaps"] > succeeded_expanded_df["current_off_snaps"])]
succeeded_def_df.to_csv("succeeded_def.csv", index=False)

succeeded_off_df = succeeded_expanded_df[(succeeded_expanded_df["current_def_snaps"] < succeeded_expanded_df["current_off_snaps"])]
succeeded_off_df.to_csv("succeeded_off.csv", index=False)

succeeded_expanded_df.to_csv("succeeded_expanded.csv", index=False)

succeeded_concat_df = pd.concat([succeeded_off_df, succeeded_def_df], ignore_index=True)
succeeded_concat_df.to_csv("succeeded_concat.csv", index=False)



positions_df = succeeded_expanded_df["current_pos"].isin(["LB", "CB", "WR", "ED"])
filtered = succeeded_expanded_df[positions_df]
filtered


,pff_player_id,player_name,prev_season,current_season,prev_position,current_pos,prev_school,current_school,wick_score,capped_wick_score,prev_def_snaps,current_def_snaps,prev_off_snaps,current_off_snaps
13,160110,Andrew Armstrong,2022,2023,WR,WR,East Texas A&M,Arkansas,70.6,70.8,1.0,NaN,667.0,711.0
14,160756,Kevin Coleman,2022,2023,WR,WR,Jackson State,Louisville,67.7,67.9,1.0,2.0,489.0,389.0
15,104059,Da'Quan Felton,2022,2023,WR,WR,Norfolk State,Virginia Tech,50.1,49.7,NaN,1.0,491.0,632.0
16,157048,Omari Kelly,2024,2025,WR,WR,Middle Tennessee,Michigan State,48.7,48.6,1.0,NaN,525.0,155.0
17,144279,Eric Rivers,2024,2025,WR,WR,Florida International,Georgia Tech,56.7,56.8,NaN,NaN,614.0,124.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126,168425,Dyoni Hill,2023,2024,CB,CB,Marshall,Miami (FL),55.9,55.9,671.0,375.0,NaN,NaN
1127,168495,Trent Henry,2023,2024,CB,CB,Grambling State,Troy,39.5,39.5,257.0,315.0,NaN,NaN
1128,172213,Dontez Fagan,2023,2024,CB,CB,Charlotte,West Virginia,57.3,57.3,628.0,528.0,NaN,NaN
1129,172259,D'Angelo Ponds,2023,2024,CB,CB,James Madison,Indiana,64.6,64.6,715.0,706.0,NaN,NaN


In [8]:
query = duckdb.query("""

SELECT prev_position prev, count(*) count1
FROM succeeded_expanded_df
GROUP BY prev
ORDER BY count1
""").to_df()
query

,prev,count1
0,C,30
1,QB,52
2,HB,56
3,G,62
4,TE,71
5,DI,80
6,T,89
7,S,94
8,LB,102
9,ED,126


In [9]:
query_2 = duckdb.query("""

SELECT prev, count1, count1/count rate
FROM query JOIN query_1 ON query.prev = query_1.prev_position
ORDER BY count1
""").to_df()
query_2

,prev,count1,rate
0,C,30,0.666667
1,QB,52,0.881356
2,HB,56,0.875000
3,G,62,0.756098
4,TE,71,0.946667
5,DI,80,0.720721
6,T,89,0.649635
7,S,94,0.734375
8,LB,102,0.836066
9,ED,126,0.823529


In [ ]:

# establish the list of ids that I need to add to an excel sheet

positions = ["WR", "LB", "ED", "CB"]

succeeded_ids_df = succeeded_expanded_df[["pff_player_id", "player_name", "prev_season"]][succeeded_expanded_df["prev_position"].isin(positions)]
# succeeded_ids_df = succeeded_ids_df[succeeded_expanded_df["prev_position"].isin(positions)]


succeeded_ids_df.to_csv("succeeded_ids.csv", index=False)

succeeded_ids_df

,pff_player_id,player_name,prev_season
13,160110,Andrew Armstrong,2022
14,160756,Kevin Coleman,2022
15,104059,Da'Quan Felton,2022
16,157048,Omari Kelly,2024
17,144279,Eric Rivers,2024
...,...,...,...
1126,168425,Dyoni Hill,2023
1127,168495,Trent Henry,2023
1128,172213,Dontez Fagan,2023
1129,172259,D'Angelo Ponds,2023


In [9]:
import sys
print(sys.executable)

c:\Users\bchm5\AppData\Local\Programs\Python\Python312\python.exe


In [10]:
import os
import json
import shutil
import asyncio
import aiohttp
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd
import time 
from dotenv import load_dotenv
load_dotenv()

# print("API Key from env:", os.environ.get("PFF_API"))

@retry(stop=stop_after_attempt(10), wait=wait_exponential(multiplier=1, min=1, max=10))
async def get_jwt_token(auth_url='https://api.profootballfocus.com/auth/login'):
    async with aiohttp.ClientSession() as session:
        data = {
            'x-api-key':os.environ['PFF_API']
        }
        async with session.post(auth_url, headers=data) as resp:
            resp.raise_for_status()
            result = await resp.json()
            return result["jwt"]

@retry(stop=stop_after_attempt(10), wait=wait_exponential(multiplier=1, min=1, max=10))
async def fetch(session, url, token_ref):

    headers = {'Authorization': f'Bearer {token_ref["token"]}'}
    async with session.get(url, timeout=10, headers=headers) as resp:
        if resp.status == 401:
            token_ref['token'] = await get_jwt_token()
        else:
            resp.raise_for_status()
        return await resp.json()
    
async def fetch_all(key, urls, max_concurrency=20):
    token = await get_jwt_token()
    token_ref = {'token':token}
    results = []
    semaphore = asyncio.Semaphore(max_concurrency)  # throttle requests
    async with aiohttp.ClientSession() as session:
        async def bound_fetch(url):
            async with semaphore:
                return await fetch(session, url, token_ref)

        tasks = [asyncio.create_task(bound_fetch(url)) for url in urls]
        for task in asyncio.as_completed(tasks):
            result = await task
            results.append(result)
    print(f"Finished {key}")
    return results

BASE_URL = "https://api.profootballfocus.com/v1/grades/ncaa/{season}/season_grade"

seasons = [2019, 2020, 2021, 2022, 2023, 2024]

urls = [BASE_URL.format(season=season) for season in seasons]
# print(urls)

results = await fetch_all("season_grades", urls)

with open("season_grades.json", "w") as f:
    json.dump(results, f, indent=2)


Finished season_grades


In [19]:
import json

with open("season_grades.json", "r") as fin:
    data = json.load(fin)

with open("season_grades_filtered.json", "a") as f:
    f.write("[\n")
    for obj in data:
        # Loop through each player in the season
        for player in obj["season_grade"]:      
            for i, v in succeeded_ids_df.iterrows():
                iter_pff_player_id = v["pff_player_id"]
                iter_prev_season = v["prev_season"]

                if player["player_id"] == iter_pff_player_id and player["season"] == iter_prev_season:
                    json.dump(player, f, indent=2)
                    f.write(",")
                    print(player)

                        
            # print(player)
        
        

# print(keep_dict)


{'position': 'ED', 'unit': 'defense', 'week': 4, 'player_id': 55198, 'season': 2020, 'run_defense_snaps': 28, 'run_defense': 66.6, 'player': 'Jordan Strachan', 'pass_rush_snaps': 25, 'pass_rush': 56.6, 'discipline': 82.1, 'defense': 60.1, 'coverage_snaps': 14, 'coverage_rank': 43, 'coverage': 54.8, 'total_snaps': 69}
{'position': 'WR', 'unit': 'offense', 'week': 4, 'player_id': 42095, 'season': 2020, 'run_block_snaps': 41, 'run_block': 50.1, 'receiving_snaps': 47, 'receiving': 59.1, 'player': 'Jahcour Pearson', 'offense_rank': 170, 'offense': 58.0, 'discipline': 77.8, 'total_snaps': 88}
{'position': 'CB', 'unit': 'defense', 'week': 4, 'player_id': 121345, 'season': 2020, 'run_defense_snaps': 33, 'run_defense': 88.0, 'run_block_snaps': 1, 'run_block': 60.0, 'player': 'Elijah Culp', 'offense': 60.0, 'discipline': 78.0, 'defense': 51.9, 'coverage_snaps': 29, 'coverage': 41.7, 'total_snaps': 97}
{'position': 'LB', 'unit': 'defense', 'week': 4, 'player_id': 63327, 'season': 2020, 'run_defen

In [33]:
import json

column_list = []


with open("season_grades_filtered.json", "r") as fin:
    data = json.load(fin)

season_grades_filtered_df = pd.DataFrame(data)
# season_grades_filtered_df

for entry in data:
    for key in entry:
        # print(key)
        if key not in column_list:
            column_list.append(key)
    # break

print(column_list)
season_grades_filtered_df

['position', 'unit', 'week', 'player_id', 'season', 'run_defense_snaps', 'run_defense', 'player', 'pass_rush_snaps', 'pass_rush', 'discipline', 'defense', 'coverage_snaps', 'coverage_rank', 'coverage', 'total_snaps', 'run_block_snaps', 'run_block', 'receiving_snaps', 'receiving', 'offense_rank', 'offense', 'defense_rank', 'run_defense_rank', 'pass_rush_rank', 'receiving_rank', 'pass_block_snaps', 'pass_block', 'run', 'run_snaps', 'pass', 'pass_snaps', 'pass_rank', 'run_block_rank', 'pass_block_rank', 'run_rank']


,position,unit,week,player_id,season,run_defense_snaps,run_defense,player,pass_rush_snaps,pass_rush,...,pass_block_snaps,pass_block,run,run_snaps,pass,pass_snaps,pass_rank,run_block_rank,pass_block_rank,run_rank
0,ED,defense,4,55198,2020,28.0,66.6,Jordan Strachan,25.0,56.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WR,offense,4,42095,2020,NaN,NaN,Jahcour Pearson,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CB,defense,4,121345,2020,33.0,88.0,Elijah Culp,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LB,defense,4,63327,2020,28.0,66.6,Zack Woodard,7.0,52.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CB,defense,4,136842,2020,49.0,61.6,Ridge Texada,14.0,49.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11110,LB,defense,8,169223,2024,102.0,54.7,Max Harris,30.0,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11111,CB,defense,8,138424,2024,117.0,43.4,Dylan Tucker,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11112,WR,offense,8,174068,2024,1.0,60.0,Jacob De Jesus,NaN,NaN,...,1.0,70.0,71.7,9.0,NaN,NaN,NaN,398.0,NaN,NaN
11113,CB,defense,8,172547,2024,188.0,69.7,Will James,1.0,47.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
agg_dict = {
    "position": "first",
    "unit": "first",
    "player": "first",

    "run_defense_snaps": "sum",
    "pass_rush_snaps": "sum",
    "coverage_snaps": "sum",
    "total_snaps": "sum",
    "run_block_snaps": "sum",
    "receiving_snaps": "sum",
    "pass_block_snaps": "sum",
    "run_snaps": "sum",
    "pass_snaps": "sum",

    "run_defense": "mean",
    "pass_rush": "mean",
    "coverage": "mean",
    "discipline": "mean",
    "defense": "mean",
    "run_block": "mean",
    "receiving": "mean",
    "pass_block": "mean",
    "run": "mean",
    "pass": "mean",
    "offense": "mean",

    "coverage_rank": "mean",
    "offense_rank": "mean",
    "defense_rank": "mean",
    "run_defense_rank": "mean",
    "pass_rush_rank": "mean",
    "receiving_rank": "mean",
    "pass_rank": "mean",
    "run_block_rank": "mean",
    "pass_block_rank": "mean",
    "run_rank": "mean"
}

df_agg = season_grades_filtered_df.groupby(["player_id", "season"]).agg(agg_dict).reset_index().sort_values(["position", "season"])

wick_combine = duckdb.query("""

SELECT *
FROM df_agg JOIN transfer_player_career_wicks ON df_agg.player_id = transfer_player_career_wicks.pff_player_id AND df_agg.season = transfer_player_career_wicks.season
ORDER BY df_agg.position
""").to_df()


# df_agg.to_csv("agged_season_stats.csv", index=False)
wick_combine.to_csv("clean_data/agged_season_stats.csv", index=False)

